In [36]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
import dalex as dx
from sklearn.model_selection import LeaveOneOut
from sklearn.naive_bayes import GaussianNB
import EDA_module
import DRSA

In [25]:
EE_015 = pd.read_csv('./data/EE_015/EE_015.csv')
EE_050 = pd.read_csv('./data/EE_050/EE_050.csv')
EE_069 = pd.read_csv('./data/EE_069/EE_069.csv')

In [26]:
EDA_module.drop_CSQ([EE_015, EE_050, EE_069])

In [27]:
df = pd.concat([EE_015, EE_050, EE_069], ignore_index=True)
df

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,FORMAT,ACMG_class,ACMG_coding_impact,...,ClippingRankSum,ExcessHet,FS,MLEAC,MLEAF,MQ,MQRankSum,QD,ReadPosRankSum,SOR
0,chr1,15820,rs2691315,G,[T],NaN,[],GT:AB:AD:AF:DP:F1R2:F2R1:SB,['Uncertain%40Significance'],['non%40coding'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chr1,17385,rs201535981,G,[A],NaN,[],GT:AB:AD:AF:DP:F1R2:F2R1:SB,['Uncertain%40Significance'],['non%40coding'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chr1,17697,rs71260069,G,[C],NaN,[],GT:AB:AD:AF:DP:F1R2:F2R1:SB,['Benign'],['non%40coding'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,chr1,133129,rs367730352,G,[A],NaN,[],GT:AB:AD:AF:DP:F1R2:F2R1:SB,['Uncertain%40Significance'],['non%40coding'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,chr1,183629,rs71267774,G,[A],NaN,[],GT:AB:AD:AF:DP:F1R2:F2R1:SB,['Benign'],['non%40coding'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444439,chrY,56856571,rs9988391,G,[A],293.77,[],GT:AB:AD:DP:GQ:PGT:PID:PL:SAC,['Uncertain%40Significance'],['non%40coding'],...,0.0,3.0103,8.315,[1],[0.5],41.25,-0.431,5.76,-1.020,0.419
444440,chrY,56856581,rs9320083,G,[A],750.77,[],GT:AB:AD:DP:GQ:PL:SAC,['Uncertain%40Significance'],['non%40coding'],...,0.0,3.0103,2.660,[1],[0.5],41.23,-2.005,16.68,-0.205,0.361
444441,chrY,56856587,rs7067511,A,[C],1164.77,[],GT:AB:AD:DP:GQ:PL:SAC,['Uncertain%40Significance'],['non%40coding'],...,0.0,3.0103,0.000,[1],[0.5],40.19,-3.699,31.48,-1.691,0.530
444442,chrY,56856668,rs5013140,A,[G],467.77,[],GT:AB:AD:DP:GQ:PGT:PID:PL:SAC,['Uncertain%40Significance'],['non%40coding'],...,0.0,3.0103,0.000,[1],[0.5],44.02,-1.952,33.41,0.183,2.985


In [28]:
df=EDA_module.drop_ACMG_AMP(df)

In [29]:
df=EDA_module.drop_Nans_threshold(df, 400000)

In [30]:
map=EDA_module.get_ACMG_mapping(df)
map

{"['Uncertain%40Significance']": 4,
 "['Benign']": 0,
 "['Likely%40Benign']": 1,
 "['Likely%40Pathogenic']": 2,
 "['Pathogenic']": 3}

In [31]:
imputer = SimpleImputer(strategy='median')
X = df.drop(columns=['ACMG_class'])
for x in X:
    X[x+"-"] = X[x]

X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
y = df['ACMG_class']


In [41]:
trainFraction = 0.7
valFraction = 0.15
testFraction = 0.15
size = df.shape[0]
idx = np.arange(size)
np.random.shuffle(idx)
trainX, trainY = X.loc[idx[:int(trainFraction*size)]].reset_index(drop=True), y[idx[:int(trainFraction*size)]]
valX = X.loc[idx[int(trainFraction*size):int((trainFraction+valFraction)*size)]].reset_index(drop=True)
valY = y[idx[int(trainFraction*size):int((trainFraction+valFraction)*size)]]
testX = X.loc[idx[int((trainFraction+valFraction)*size):]].reset_index(drop=True)
testY = y[idx[int((trainFraction+valFraction)*size):]]

In [44]:
drsa_params=([1]*53)+([-1]*53)
drsa = DRSA.DRSA(drsa_params, credibility=0.9)
drsa.fit(trainX, trainY.reset_index(drop=True))
pred2 = drsa.predict(valX)

class <= 0
class <= 1
class <= 2
class <= 3
class >= 1
class >= 2
class >= 3
class >= 4
('gnomadGenomes_AF-', <built-in function ge>, 3015.0)
268758 298339
Class <=0 if gnomadGenomes_AF- >= 3015.0 268758 298339
('FILTER-', <built-in function ge>, 66.0)
261878 288706
Class <=0 if FILTER- >= 66.0 261878 288706
('gnomadGenomes_AF_ethnic-', <built-in function ge>, 3600.0)
268435 297659
Class <=0 if gnomadGenomes_AF_ethnic- >= 3600.0 268435 297659
('FILTER-', <built-in function ge>, 64.0)
263186 290614
Class <=0 if FILTER- >= 64.0 263186 290614
('gnomadGenomesAN-', <built-in function ge>, 8033.0)
262061 288140
Class <=0 if gnomadGenomesAN- >= 8033.0 262061 288140
5
4
('ECNT', <built-in function ge>, 27.0)
830 852
Class >=1 if ECNT >= 27.0 830 852
('gnomadGenomes_AF_ethnic-', <built-in function le>, 7060.0)
14915 16269
Class >=1 if gnomadGenomes_AF_ethnic- <= 7060.0 14915 16269
('gnomadGenomesEthnic_AC_Hom-', <built-in function le>, 0.0)
27866 31346
('GERMQ', <built-in function ge>, 31.0)
26

KeyboardInterrupt: 